# Create an AgentCore Gateway and connect it to a Lambda Target
**Description**: This notebook shows how to create an AgentCore Gateway and add a Lambda Target. Gateway then allows agents to use the tools defined in Lambda as an MCP server.

#### Imports

In [1]:
import json
import time
import boto3

### Configuration Parameters

In [2]:
region = 'us-east-1'
gateway_client = boto3.client('bedrock-agentcore-control', region_name=region)

#### Cognito/Auth Configurations

In [3]:
cognito_client_id = ''
cognito_discovery_url = ''
auth_config = {"customJWTAuthorizer": {"allowedClients": [cognito_client_id], "discoveryUrl": cognito_discovery_url}}
agentcore_gateway_iam_role = ''

### Create AgentCore Gateway

In [4]:
create_response = gateway_client.create_gateway(
    name='MCP-tech-talk-travel',
    roleArn = agentcore_gateway_iam_role,
    protocolType='MCP',
    protocolConfiguration={
        'mcp': {
            'supportedVersions': ['2025-03-26'],
            'searchType': 'SEMANTIC'
        }
    },
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway with AWS Lambda target type to demo access controls for a travel agent.'
)

time.sleep(10)
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(f'Gateway ID:  {gatewayID}')
print(f'Gateway URL: {gatewayURL}')

Gateway ID:  mcp-tech-talk-travel-po9gnocfuh
Gateway URL: https://mcp-tech-talk-travel-po9gnocfuh.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp


### Create AgentCore Gateway Target for Travel Tools

In [6]:
lambda_role_arn = ''
lambda_function_arn = ''
lambda_tool_schema = json.load(open('tech_talk_travel_target_schema.json', 'r'))
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_function_arn,
             "toolSchema": lambda_tool_schema
        }
    }
}
lambda_credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
lambda_target_name = 'TravelTools'

if True:
    response = gateway_client.create_gateway_target(
        gatewayIdentifier=gatewayID,
        name=lambda_target_name,
        description='Target for Tech Talk Travel',
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=lambda_credential_config)